In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor,Resize,Compose
from torch.utils.data.sampler import SubsetRandomSampler


from sklearn import metrics

import time
import os
import pickle

import matplotlib.pyplot as plt

from src.train_valid import train, validation
from src.Basic_CNN_Architecture import ResNet, ResidualBlock
from src.data_loader import load_dog_data,SquarePadding
from src.utils import unravel_image

In [2]:
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

    

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2060 SUPER


In [4]:
#val_df_path =
#test_df_path =
#root_dir =



#Put your data path here
data_path = r'C:\Users\Alec\Documents\GitHub\IncubatorCVProject\DogsDataset\images\Images'

In [6]:
###################################
############ load data ############
###################################

# after feature engineering(task 3)

#Variables for splitting the dataset into train/test
valid_split = .1
test_split = .1
batch_size = 16
shuffle_dataset = True
random_seed = 42

#Load initial dataset
dog_dataset = ImageFolder(data_path,transform=Compose([
    SquarePadding(),
    Resize((128,128)),
    ToTensor()
])) 
# Split 
dataset_size = len(dog_dataset)
indices = list(range(dataset_size))
split_idx1 = int(np.floor((valid_split+test_split) * dataset_size))
split_idx2 = int(np.floor(test_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    
test_indices, valid_indices, train_indices = indices[:split_idx2], indices[split_idx2:split_idx1], indices[split_idx1:]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)
test_sampler = SubsetRandomSampler(test_indices)

#Load Train, valid, test          
train_loader = torch.utils.data.DataLoader(dog_dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(dog_dataset, batch_size=batch_size,
                                                sampler=valid_sampler)
test_loader =  torch.utils.data.DataLoader(dog_dataset, batch_size=batch_size,
                                                sampler=test_sampler)



In [ ]:
###################################
############ Train ############
###################################
print('###################################')
print('############## Train ##############')
print('###################################')

# training process
# to be finished later
model = ResNet(
    ResidualBlock,
    in_channels = 3,
    conv1_out_channels = 64,
    conv2_out_channels = 128,
    num_blocks = 20,
    lin1_out_channels = 128,
    num_classes=len(dog_dataset.classes),
    kernel_size = 5,
    stride = 2,
    padding = 2
                        )
model.to(device)

def train_valid(optimizer = optim.Adam(model.parameters()), epochs = 20, model = model,
                train_criterion = nn.CrossEntropyLoss(), train_loader = train_loader,
                valid_criterion = nn.CrossEntropyLoss(), valid_loader = valid_loader,
                device = device):

    start_epoch = 1
    #or: best_val_acc = 0
    best_val_loss = np.inf

    history = {"train_loss":[], "train_acc":[],
                "valid_loss":[], "valid_acc":[], "valid_preds_list":[],
                "valid_truelabels_list":[], "valid_probas_list":[], "valid_auc_score":[]}

    start_time = time.time()

    for epoch in range(start_epoch, epochs + 1):

        train_loss, train_acc = train(epoch, model, optimizer, train_criterion, train_loader, device)
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)

        print('epoch: ', epoch)
        print('{}: loss: {:.4f} acc: {:.4f}'.format('training', train_loss, train_acc))

        valid_loss, valid_acc, valid_preds_list, valid_truelabels_list, valid_probas_list, valid_auc_score = validation(epoch, model, optimizer, valid_criterion, valid_loader, device)
        history["valid_loss"].append(valid_loss)
        history["valid_acc"].append(valid_acc)
        history["valid_preds_list"].append(valid_preds_list)
        history["valid_truelabels_list"].append(valid_truelabels_list)
        history["valid_probas_list"].append(valid_probas_list)
        history["valid_auc_score"].append(valid_auc_score)

        print('{}: loss: {:.4f} acc: {:.4f} auc: {:.4f}\n'.format('validation', valid_loss, valid_acc, valid_auc_score))


    return history

results = train_valid()



###################################
############## Train ##############
###################################
epoch:  1
training: loss: 4.7932 acc: 0.0085
validation: loss: 4.7956 acc: 0.0058 auc: 0.5000

epoch:  2
training: loss: 4.7931 acc: 0.0086
validation: loss: 4.7959 acc: 0.0058 auc: 0.5000

epoch:  3
training: loss: 4.7931 acc: 0.0086
validation: loss: 4.7956 acc: 0.0058 auc: 0.5000

epoch:  4
training: loss: 4.7931 acc: 0.0086
validation: loss: 4.7959 acc: 0.0058 auc: 0.5000

epoch:  5
training: loss: 4.7931 acc: 0.0086
validation: loss: 4.7959 acc: 0.0058 auc: 0.5000

epoch:  6
training: loss: 4.7931 acc: 0.0086
validation: loss: 4.7959 acc: 0.0058 auc: 0.5000

epoch:  7
training: loss: 4.7931 acc: 0.0086
validation: loss: 4.7959 acc: 0.0058 auc: 0.5000

epoch:  8
training: loss: 4.7931 acc: 0.0086
validation: loss: 4.7959 acc: 0.0058 auc: 0.5000

epoch:  9
training: loss: 4.7931 acc: 0.0086
validation: loss: 4.7959 acc: 0.0058 auc: 0.5000

epoch:  10
training: loss: 4.7931 acc